In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


class Generator(nn.Module):
    def __init__(self, classes, channels, img_size, latent_dim):
        super(Generator, self).__init__()
        self.classes = classes
        self.channels = channels
        self.img_size = img_size
        self.latent_dim = latent_dim
        self.img_shape = (self.channels, self.img_size, self.img_size)
        self.label_embedding = nn.Embedding(self.classes, self.classes)

        self.model = nn.Sequential(
            # 1st layer
            nn.Linear(in_features = self.latent_dim + self.classes, out_features = 64),
            nn.LeakyReLU(),
            # 2nd layer
            nn.Linear(in_features = 64, out_features = 128),
            nn.LeakyReLU(),
            # 3rd layer
            nn.BatchNorm1d(num_features = 128, momentum = 0.8),
            # 4th layer
            nn.Linear(in_features = 128, out_features = 64),
            nn.LeakyReLU(),
            # 5th layer
            nn.Linear(in_features = 64, out_features = int(np.prod(self.img_shape)))
        )

    def forward(self, noise, labels):
        z = torch.cat((self.label_embedding(labels), noise), -1)
        x = self.model(z)
        x = x.view(x.size(0), *self.img_shape)
        return x


class Discriminator(nn.Module):
    def __init__(self, classes, channels, img_size, latent_dim):
        super(Discriminator, self).__init__()
        self.classes = classes
        self.channels = channels
        self.img_size = img_size
        self.latent_dim = latent_dim
        self.img_shape = (self.channels, self.img_size, self.img_size)
        self.label_embedding = nn.Embedding(self.classes, self.classes)
        self.adv_loss = torch.nn.BCELoss()

        self.model = nn.Sequential(
            # 1st layer
            nn.Linear(in_features = self.classes + int(np.prod(self.img_shape)), out_features = 64),
            nn.LeakyReLU(),
            # 2nd layer
            nn.Linear(in_features = 64, out_features = 128),
            nn.LeakyReLU(),
            # 3rd layer
            nn.Linear(in_features = 128, out_features = 128),
            nn.LeakyReLU(),
            # 4th layer
            nn.Linear(in_features = 128, out_features = 64),
            nn.LeakyReLU(),
            # 5th layer
            nn.Dropout(p = 0.4),
            # 6th layer
            nn.Linear(in_features = 64, out_features = 1),
            nn.Sigmoid()
        )

    def forward(self, image, labels):
        x = torch.cat((image.view(image.size(0), -1), self.label_embedding(labels)), -1)
        display(x.shape)
        return self.model(x)

    def loss(self, output, label):
        return self.adv_loss(output, label)

In [2]:
import errno
import os
import shutil
import sys


def to_np(var):
    """Exports torch.Tensor to Numpy array.
    """
    return var.detach().cpu().numpy()


def create_folder(folder_path):
    """Create a folder if it does not exist.
    """
    try:
        os.makedirs(folder_path)
    except OSError as _e:
        if _e.errno != errno.EEXIST:
            raise


def clear_folder(folder_path):
    """Clear all contents recursively if the folder exists.
    Create the folder if it has been accidently deleted.
    """
    create_folder(folder_path)
    for the_file in os.listdir(folder_path):
        _file_path = os.path.join(folder_path, the_file)
        try:
            if os.path.isfile(_file_path):
                os.unlink(_file_path)
            elif os.path.isdir(_file_path):
                shutil.rmtree(_file_path)
        except OSError as _e:
            print(_e)


class StdOut(object):
    """Redirect stdout to file, and print to console as well.
    """
    def __init__(self, output_file):
        self.terminal = sys.stdout
        self.log = open(output_file, "a")

    def write(self, message):
        self.terminal.write(message)
        self.terminal.flush()
        self.log.write(message)
        self.log.flush()

    def flush(self):
        self.terminal.flush()
        self.log.flush()


def boolean_string(s):
    if s not in {'False', 'True'}:
        raise ValueError('Not a valid boolean string')
    return s == 'True'

In [3]:
import itertools
import os
import time

from datetime import datetime

import numpy as np
import torch
import torchvision.utils as vutils


class Model(object):
    def __init__(self,
                 name,
                 device,
                 data_loader,
                 classes,
                 channels,
                 img_size,
                 latent_dim,
                 style_dim=2):
        self.name = name
        self.device = device
        self.data_loader = data_loader
        self.classes = classes
        self.channels = channels
        self.img_size = img_size
        self.latent_dim = latent_dim
        self.style_dim = style_dim
        self.netG = Generator(self.classes, self.channels, self.img_size, self.latent_dim)
        self.netG.to(self.device)
        self.netD = Discriminator(self.classes, self.channels, self.img_size, self.latent_dim)
        self.netD.to(self.device)
        self.optim_G = None
        self.optim_D = None
        self.optim_info = None

    @property
    def generator(self):
        return self.netG

    @property
    def discriminator(self):
        return self.netD

    def create_optim(self, lr, alpha=0.5, beta=0.999):
        self.optim_G = torch.optim.Adam(filter(lambda p: p.requires_grad,
                                        self.netG.parameters()),
                                        lr=lr,
                                        betas=(alpha, beta))
        self.optim_D = torch.optim.Adam(filter(lambda p: p.requires_grad,
                                        self.netD.parameters()),
                                        lr=lr,
                                        betas=(alpha, beta))

    def _to_onehot(self, var, dim):
        res = torch.zeros((var.shape[0], dim), device=self.device)
        res[range(var.shape[0]), var] = 1.
        return res

    def train(self,
              epochs,
              log_interval=100,
              out_dir='',
              verbose=True):
        self.netG.train()
        self.netD.train()
        viz_z = torch.zeros((self.data_loader.batch_size, self.latent_dim), device=self.device)
        viz_noise = torch.randn(self.data_loader.batch_size, self.latent_dim, device=self.device)
        nrows = self.data_loader.batch_size // 8
        viz_label = torch.LongTensor(np.array([num for _ in range(nrows) for num in range(8)])).to(self.device)
        viz_onehot = self._to_onehot(viz_label, dim=self.classes)
        viz_style = torch.zeros((self.data_loader.batch_size, self.style_dim), device=self.device)
        total_time = time.time()
        for epoch in range(epochs):
            batch_time = time.time()
            for batch_idx, (data, target) in enumerate(self.data_loader):

                # Old code
                #data, target = data.to(self.device), target.to(self.device)

                # New code
                #"""
                data = torch.stack(data)
                data = torch.tensor(data, dtype = torch.float32)
                target = torch.tensor(target)
                data, target = data.T.to(self.device), target.to(self.device)
                #"""

                batch_size = data.size(0)
                real_label = torch.full((batch_size, 1), 1., device=self.device)
                fake_label = torch.full((batch_size, 1), 0., device=self.device)

                # Train G
                self.netG.zero_grad()
                z_noise = torch.randn(batch_size, self.latent_dim, device=self.device)
                x_fake_labels = torch.randint(0, self.classes, (batch_size,), device=self.device)
                x_fake = self.netG(z_noise, x_fake_labels)
                display('x_fake', x_fake.shape, 'x_fake_labels', x_fake_labels.shape)
                y_fake_g = self.netD(x_fake, x_fake_labels)
                g_loss = self.netD.loss(y_fake_g, real_label)
                g_loss.backward()
                self.optim_G.step()

                # Train D
                self.netD.zero_grad()
                display('data', data.shape, 'target', target.shape)
                y_real = self.netD(data, target)
                d_real_loss = self.netD.loss(y_real, real_label)

                y_fake_d = self.netD(x_fake.detach(), x_fake_labels)
                d_fake_loss = self.netD.loss(y_fake_d, fake_label)
                d_loss = (d_real_loss + d_fake_loss) / 2
                d_loss.backward()
                self.optim_D.step()

                if verbose and batch_idx % log_interval == 0 and batch_idx > 0:
                    print('Epoch {} [{}/{}] loss_D: {:.4f} loss_G: {:.4f} time: {:.2f}'.format(
                            epoch, batch_idx, len(self.data_loader),
                            d_loss.mean().item(),
                            g_loss.mean().item(),
                            time.time() - batch_time))
                    vutils.save_image(data, os.path.join(out_dir, 'real_samples.png'), normalize=True)
                    with torch.no_grad():
                        viz_sample = self.netG(viz_noise, viz_label)
                        vutils.save_image(viz_sample, os.path.join(out_dir, 'fake_samples_{}.png'.format(epoch)), nrow=8, normalize=True)
                    batch_time = time.time()

            self.save_to(path=out_dir, name=self.name, verbose=False)
        if verbose:
            print('Total train time: {:.2f}'.format(time.time() - total_time))

    def save_to(self,
                path='',
                name=None,
                verbose=True):
        if name is None:
            name = self.name
        if verbose:
            print('\nSaving models to {}_G.pt and {}_D.pt ...'.format(name, name))
        torch.save(self.netG.state_dict(), os.path.join(path, '{}_G.pt'.format(name)))
        torch.save(self.netD.state_dict(), os.path.join(path, '{}_D.pt'.format(name)))

In [4]:
import argparse
import os
import sys

import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms

import pandas as pd
from preprocessing import remove_incomplete_days
from sklearn.preprocessing import minmax_scale


def main(batch_size, model, classes, channels, img_size, latent_dim, lr, epochs, log_interval, out_dir):
    device = torch.device('cuda:0')

    # Original dataset
    dataset = dset.MNIST(root = '~/Data/mnist', download = True, transform = transforms.Compose([transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]))
    dataloader = torch.utils.data.DataLoader(dataset, batch_size = batch_size, shuffle = True, num_workers = 4, pin_memory = True)
    return dataset, dataloader

    # New dataset (Enercoop)
    """
    df_loadProfiles = pd.read_parquet(r'data/load_profiles.parquet.gzip')
    df_loadProfiles = remove_incomplete_days(df_loadProfiles)
    df_temp = df_loadProfiles.pivot_table(values = '1163', index = ['date', 'month of the year', 'day off'], columns = 'hour of the day').reset_index()
    labels = list((df_temp['month of the year'].astype(str) + df_temp['day off'].astype(str)).astype(int))
    df_temp.drop(['date', 'month of the year', 'day off'], axis = 1, inplace = True)
    df_temp = pd.DataFrame(minmax_scale(df_temp, feature_range = (-0.5, 0.5), axis = 1))
    timeSeries = df_temp.values.tolist()
    dataloader = torch.utils.data.DataLoader([[timeSeries[i], labels[i]] for i in range(len(labels))], shuffle = True, batch_size = batch_size)
    """

    model = Model(model, device, dataloader, classes, channels, img_size, latent_dim)
    model.create_optim(lr)

    # Train
    model.train(epochs, log_interval, out_dir, True)

    model.save_to('')

#main(batch_size = 128, model = 'cgan', classes = 10, channels = 1, img_size = 64, latent_dim = 100, lr = 0.002, epochs = 2, log_interval = 100, out_dir = 'output')

In [5]:
#main(batch_size = 100, model = 'cgan', classes = 24, channels = 1, img_size = 24, latent_dim = 20, lr = 0.0004, epochs = 2, log_interval = 100, out_dir = 'output')
dataset, dataloader = main(batch_size = 128, model = 'cgan', classes = 10, channels = 1, img_size = 64, latent_dim = 100, lr = 0.002, epochs = 2, log_interval = 100, out_dir = 'output')

In [8]:
import pandas as pd
from preprocessing import remove_incomplete_days
from sklearn.preprocessing import minmax_scale
import torch

df_loadProfiles = pd.read_parquet(r'data/load_profiles.parquet.gzip')
df_loadProfiles = remove_incomplete_days(df_loadProfiles)
df_temp = df_loadProfiles.pivot_table(values = '1163', index = ['date', 'month of the year', 'day off'], columns = 'hour of the day').reset_index()
labels = list((df_temp['month of the year'].astype(str) + df_temp['day off'].astype(str)).astype(int))
df_temp.drop(['date', 'month of the year', 'day off'], axis = 1, inplace = True)
df_temp = pd.DataFrame(minmax_scale(df_temp, feature_range = (-0.5, 0.5), axis = 1))
timeSeries = df_temp.values.tolist()
trainloader = torch.utils.data.DataLoader([[timeSeries[i], labels[i]] for i in range(len(labels))], shuffle = True, batch_size = 100)